# 4 - Análise do município

Também é interessante entender como a demanda e o atendimento se comportaram no município de São Paulo como um todo. Para isso, vamos importar os dados e processá-los, tratando o município da mesma maneira como os distritos foram.

In [1]:
import pandas as pd

dados_educ_pop = pd.read_csv('dados/dados_educ_pop.csv', sep = ',')

Primeiro, devemos unificar todos os distritos e guardar essa informação.

In [2]:
dados_matricula = dados_educ_pop.groupby(['MES_ANO_REF'])['MAT_CRECHE'].sum().to_frame()
dados_demanda = dados_educ_pop.groupby(['MES_ANO_REF'])['DEM_CRECHE'].sum().to_frame()
dados_populacao = dados_educ_pop.groupby(['MES_ANO_REF'])['TOTAL'].sum().to_frame()

dados_municipio = dados_matricula.merge(dados_demanda, on = ['MES_ANO_REF'])
dados_municipio = dados_municipio.merge(dados_populacao, on = ['MES_ANO_REF'])

Precisamos que os dados fiquem ordenados por mês e ano, para que o gráfico possa ser feito. Antes disso, é necessário deixar o mês e o ano em um formato numérico.

In [3]:
meses = {"mar" : 3, "jun" : 6, "set" : 9, "dez" : 12}

for i in dados_matricula.index:
    dados_municipio.at[i, 'MES'] = int(meses.get(i[:3]))
    dados_municipio.at[i, 'ANO'] = int(i[4:]) + 2000
    
dados_municipio = dados_municipio.sort_values(by = ['ANO','MES'], ascending=True)

Com isso, é possível gerar o gráfico.

In [4]:
import matplotlib.pyplot as plt

fig = plt.figure(dpi=100, figsize=(9, 5))
plt.title('Evolução do atendimento das creches em São Paulo')
plt.plot(dados_municipio.index, dados_municipio['MAT_CRECHE'].values, label='Matrículas nas creches do município', color='#33638D')
plt.plot(dados_municipio.index, dados_municipio['DEM_CRECHE'].values, label='Demanda no distrito', color='#73D055')
plt.plot(dados_municipio.index, dados_municipio['TOTAL'].values, label='Total da população no distrito', color='#440154')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2),  shadow=True, ncol=2)
plt.ylabel('Crianças de 0 a 3 anos')
plt.xlabel('Data de Referência')
p = plt.xticks(rotation=90)
plt.savefig('graficos/sao_paulo.png', bbox_inches='tight')
plt.close(fig)

Podemos também calcular a evolução do atendimento e a evolução da fila.

In [20]:
ev_atend_abs = dados_municipio.at['dez-17', 'MAT_CRECHE'] - dados_municipio.at['dez-06', 'MAT_CRECHE']
ev_atend_rel = dados_municipio.at['dez-17', 'MAT_CRECHE'] / dados_municipio.at['dez-06', 'MAT_CRECHE'] * 100
ev_dem_abs = dados_municipio.at['dez-17', 'DEM_CRECHE'] - dados_municipio.at['dez-06', 'DEM_CRECHE']
ev_dem_rel = dados_municipio.at['dez-17', 'DEM_CRECHE'] / dados_municipio.at['dez-06', 'DEM_CRECHE'] * 100

# f'O atendimento variou em {int(ev_atend_abs)} matrículas, e é, em dezembro de 2017, {round(ev_atend_rel, 2)}% do que era em dezembro de 2006'
# f'A demanda variou em {int(ev_dem_abs)} matrículas, e é, em dezembro de 2017, {round(ev_dem_rel, 2)}% do que era em dezembro de 2006'

'A demanda variou em -85502 matrículas, e é, em dezembro de 2017, 34.02% do que era em dezembro de 2006'